# Projet M2 MIAS : Risque de réidentification (Février 2026)

**Auteur :** [Votre Nom]
**Superviseur :** Pr Emmanuel Chazard

## 1. Objectif du projet
Conformément au sujet, l'objectif est d'imaginer et tester des indicateurs quantitatifs du risque de réidentification sur des bases de données de santé.

Nous analysons les 3 critères du G29 (CNIL) :
1.  **Individualisation** : Peut-on isoler un individu ?
2.  **Inférence** : Peut-on déduire une information sensible ?
3.  **Corrélation** : Peut-on lier des ensembles de données (ou des attributs entre eux) ?

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Configuration graphique
plt.style.use('seaborn-v0_8')

## 2. Chargement des Données

Chargement de la base de référence et des fichiers variants (`out_direct` et `out_sample`) depuis le répertoire `projets_donnees`.

In [ ]:
# Chemin vers le répertoire des données
DATA_DIR = "projets_donnees"

def load_data(directory):
    data = {}
    
    # 1. Chargement de la base de référence
    ref_path = os.path.join(directory, "connaissances_externes.txt")
    if os.path.exists(ref_path):
        data['reference'] = pd.read_csv(ref_path, sep=None, engine='python')
        print(f"Base référence chargée : {len(data['reference'])} lignes")
    else:
        raise FileNotFoundError(f"Le fichier {ref_path} est introuvable.")

    # 2. Chargement des fichiers variants
    pattern = os.path.join(directory, "out_*.txt")
    files = glob.glob(pattern)
    
    if not files:
        print("Attention : Aucun fichier 'out_*.txt' trouvé.")

    for filepath in files:
        filename = os.path.basename(filepath)
        name_key = filename.replace('.txt', '') # ex: out_direct_10
        try:
            df = pd.read_csv(filepath, sep=None, engine='python')
            data[name_key] = df
            print(f" -> Chargé : {name_key} ({len(df)} lignes)")
        except Exception as e:
            print(f"Erreur lors du chargement de {filename} : {e}")
            
    return data

# Exécution du chargement
datasets = load_data(DATA_DIR)
df_reference = datasets['reference']

# Aperçu
df_reference.head()

## 3. Définition des Indicateurs (G29)

Nous définissons ici les trois indicateurs demandés.

### 3.1 Individualisation (Réidentification)
Capacité à isoler un enregistrement unique dans la base publiée et à le lier correctement à la base de connaissances.

### 3.2 Inférence (Précision)
Capacité à déduire une valeur sensible (ex: `diabete`) avec exactitude, même si l'identification formelle a échoué.

### 3.3 Corrélation (Conservation de structure)
Le mélange des colonnes (shuffling) vise à briser les liens entre les variables (ex: lien entre `Age` et `Pathologie`).
* **Mesure** : Nous calculons la matrice de corrélation de la base de référence et celle de la base attaquée. Le risque de corrélation est élevé si les matrices sont similaires (la structure des liens est préservée).
* **Formule** : Nous utilisons une similarité basée sur la différence moyenne des coefficients de corrélation (Pearson/Spearman).

In [ ]:
def calculate_correlation_risk(df_ref, df_anon):
    """
    Calcule le risque de corrélation en comparant les matrices de corrélation.
    Retourne un score de 0 à 100% (100% = structure parfaitement conservée).
    """
    # 1. Sélection et encodage des colonnes communes pour la corrélation
    common_cols = [c for c in df_ref.columns if c in df_anon.columns and c != 'id_sejour']
    
    if not common_cols:
        return 0.0
    
    # Pour gérer les variables catégorielles, on les factorise (encodage simple)
    # Cela permet de voir si les liens statistiques sont maintenus
    def get_encoded_matrix(df, cols):
        temp_df = pd.DataFrame()
        for c in cols:
            if df[c].dtype == 'object':
                temp_df[c] = pd.factorize(df[c])[0]
            else:
                temp_df[c] = df[c]
        return temp_df.corr(method='spearman') # Spearman gère mieux les relations non linéaires/rangs

    corr_ref = get_encoded_matrix(df_ref, common_cols)
    corr_anon = get_encoded_matrix(df_anon, common_cols)
    
    # 2. Calcul de la différence moyenne absolue entre les matrices
    # On remplace les NaN par 0 (cas de variance nulle)
    diff_matrix = (corr_ref.fillna(0) - corr_anon.fillna(0)).abs()
    mean_diff = diff_matrix.mean().mean()
    
    # 3. Score de risque : Si diff = 0, Risque = 100%. Si diff est grand, Risque diminue.
    # Une différence moyenne de 0 signifie une corrélation parfaite avec l'original.
    # Empiriquement, si la différence > 0.5, la structure est très cassée.
    risk_score = max(0, (1 - mean_diff * 2)) * 100
    
    return risk_score

def calculate_risk_metrics(df_connaissance, df_publie, quasi_identifiers, sensitive_col='diabete'):
    """
    Calcule les 3 indicateurs : Individualisation, Inférence, Corrélation.
    """
    # --- 1. Indicateur Corrélation (Global sur le fichier) ---
    # Indépendant du scénario de QI, dépend de l'intégrité globale du fichier
    risk_corr = calculate_correlation_risk(df_connaissance, df_publie)

    # --- Préparation pour Individualisation/Inférence ---
    missing_cols = [c for c in quasi_identifiers if c not in df_publie.columns]
    if missing_cols:
        return 0.0, 0.0, risk_corr

    # Isoler les combinaisons uniques
    counts = df_publie.groupby(quasi_identifiers).size()
    unique_combinations = counts[counts == 1].index
    
    if len(unique_combinations) == 0:
        return 0.0, 0.0, risk_corr
    
    df_pub_unique = df_publie.set_index(quasi_identifiers)
    df_pub_unique = df_pub_unique[df_pub_unique.index.isin(unique_combinations)].reset_index()
    
    # Attaque par jointure
    merged = pd.merge(
        df_pub_unique, 
        df_connaissance, 
        on=quasi_identifiers, 
        how='inner', 
        suffixes=('_pub', '_know')
    )
    
    if len(merged) == 0:
        return 0.0, 0.0, risk_corr
        
    # --- 2. Indicateur Individualisation ---
    reid_success = merged[merged['id_sejour_pub'] == merged['id_sejour_know']]
    risk_reid = len(reid_success) / len(df_publie) * 100
    
    # --- 3. Indicateur Inférence ---
    if sensitive_col in df_publie.columns and sensitive_col in df_connaissance.columns:
        inference_success = merged[merged[f'{sensitive_col}_pub'] == merged[f'{sensitive_col}_know']]
        risk_inference = len(inference_success) / len(merged) * 100
    else:
        risk_inference = 0.0
    
    return risk_reid, risk_inference, risk_corr

## 4. Exécution des Scénarios

Nous testons les scénarios "Faible", "Moyen" et "Fort" pour observer l'impact sur les 3 indicateurs.

In [ ]:
scenarios = {
    '1_Faible': ['age10', 'sexe', 'entree_date_y'],
    '2_Moyen': ['age5', 'sexe', 'entree_date_ym', 'specialite'],
    '3_Fort': ['age', 'sexe', 'entree_date_ymd', 'specialite', 'entree_mode', 'chirurgie']
}

results = []

for name, df_var in datasets.items():
    if name == 'reference': continue
    
    parts = name.split('_')
    if len(parts) >= 3:
        file_type = 'Direct' if 'direct' in name else 'Sample'
        try: shuffle_level = int(parts[-1])
        except: shuffle_level = 0
    else: continue

    for scen_name, cols in scenarios.items():
        r_reid, r_inf, r_corr = calculate_risk_metrics(df_reference, df_var, cols, sensitive_col='diabete')
        
        results.append({
            'Type': file_type,
            'Mélange (%)': shuffle_level,
            'Scénario': scen_name,
            'Individualisation (%)': r_reid,
            'Inférence (%)': r_inf,
            'Corrélation (%)': r_corr
        })

df_res = pd.DataFrame(results)
df_res.head()

## 5. Visualisation et Analyse

Nous affichons les courbes pour les 3 critères.

In [ ]:
def plot_all_metrics(df_res):
    if df_res.empty:
        print("Pas de données à afficher.")
        return

    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    # 1. Individualisation
    sns.lineplot(data=df_res, x='Mélange (%)', y='Individualisation (%)', 
                 hue='Scénario', style='Type', markers=True, ax=axes[0])
    axes[0].set_title("Risque d'Individualisation")
    axes[0].set_ylim(-5, 105)

    # 2. Inférence
    sns.lineplot(data=df_res, x='Mélange (%)', y='Inférence (%)', 
                 hue='Scénario', style='Type', markers=True, ax=axes[1])
    axes[1].set_title("Risque d'Inférence (Qualité de la donnée déduite)")
    axes[1].set_ylim(-5, 105)

    # 3. Corrélation (Ne dépend pas du scénario QI, on moyenne ou on prend un scénario)
    # On filtre sur un scénario pour éviter la superposition inutile (la métrique est globale)
    subset = df_res[df_res['Scénario'] == list(scenarios.keys())[0]]
    sns.lineplot(data=subset, x='Mélange (%)', y='Corrélation (%)', 
                 style='Type', markers=True, ax=axes[2], color='green')
    axes[2].set_title("Risque de Corrélation (Conservation de structure)")
    axes[2].set_ylim(-5, 105)

    plt.tight_layout()
    plt.show()

plot_all_metrics(df_res)

### Interprétation des résultats

1.  **Individualisation** : Devrait chuter drastiquement avec le mélange. Le fichier `Sample` présente généralement un risque plus faible grâce à l'incertitude du tirage.
2.  **Inférence** : Même si l'individualisation est faible, l'inférence peut rester correcte autour de 50% (hasard) ou plus si la distribution de la variable sensible est déséquilibrée (ex: 90% non-diabétique), mais le mélange la dégrade.
3.  **Corrélation** : C'est l'indicateur le plus sensible au mélange par colonne. 
    * À 0% de mélange, le risque est de 100% (les liens Âge <-> Pathologie sont intacts).
    * Dès que le mélange augmente, la structure de la base s'effondre, rendant impossible l'analyse statistique fiable (perte d'utilité majeure).